In [2]:
!python -m pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 27.0 MB/s eta 0:00:00m eta 0:00:010:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.6/346.6 KB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 505.5/505.5 KB 42.1 MB/s eta 0:00:00


In [3]:
import json
import numpy as np
import pandas as pd
from time import time

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier

from sklearn.metrics import accuracy_score
import pickle 

/tmp/ipykernel_53429/923370504.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [51]:
name_model = "mlp_rnd_walk_dist-mini_publaynet_50"
with open(f"./dataset/json/{name_model}.json", "r") as f:
    dataset = json.load(f)

In [14]:
print("Первый x:\n", dataset["x"][0])
print("Последний, x:\n", dataset["x"][-1])
print("Размер x:\t", len(dataset["x"][0]))
print("Первый y:\t", dataset["y"][0])
print("Последний y:\t", dataset["y"][0])
print("Кол-во записей:\t", len(dataset["x"]))

Первый x:
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.017167381974248927, 0.017167381974248927, 0.017167381974248927, 0.017167381974248927, 0.017167381974248927, 0.0, 0.02145922746781116, 0.017167381974248927, 0.06437768240343347, 0.012875536480686695, 0.0, 0.0815450643776824, 0.008583690987124463, 0.017167381974248927, 0.017167381974248927, 0.0, 0.0, 0.017167381974248927, 0.017167381974248927, 0.008583690987124463, 0.0, 0.008583690987124463, 0.017167381974248927, 0.02145922746781116, 0.008583690987124463, 0.06437768240343347, 0.017167381974248927, 0.0, 0.017167381974248927, 0.0, 0.06437768240343347, 0.012875536480686695, 0.06437768240343347, 0.012875536480686695, 0.02145922746781116, 0.0, 0.02145922746781116, 0.017167381974248927, 0.034334763948497854, 0.012875536480686695, 0.02145922746781116, 0.06437768240343347, 0.06437768240343347, 0.0, 0.06437768240343347]
Последний, x:
 [0.0, 0.0, 0.03303303303303303, 0.015015015015015015, 0.015015015015015015, 0.012012012012012012, 0.012012012012012012, 0.012

In [34]:

X = np.array(dataset["x"])
y = np.array(dataset["y"])


In [35]:
# y = y[X.sum(1)!=0]
# X = X[X.sum(1)!=0, :]


In [36]:

my_norm = lambda x: x if x.sum() == 0 else x/x.sum()
X = np.array([my_norm(x) for x in X])


In [37]:
len(y), len(X)

(4997, 4997)

In [38]:
keys = np.unique(y)
Y_keys = dict()
for i, key in enumerate(keys):
    zeros = np.zeros((len(keys)))
    zeros[i] = 1
    Y_keys[key] = zeros
Y_keys
Y = np.array([Y_keys[yi] for yi in y] )
Y_keys, Y

({1: array([1., 0., 0., 0., 0.]),
  2: array([0., 1., 0., 0., 0.]),
  3: array([0., 0., 1., 0., 0.]),
  4: array([0., 0., 0., 1., 0.]),
  5: array([0., 0., 0., 0., 1.])},
 array([[1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        ...,
        [0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0.]]))

In [39]:
X_clus_list = []
Y_clus_list = []
for i, item in enumerate(Y_keys.keys()):
    X_clus_list.append(X[Y[:,i]==1,:])
    Y_clus_list.append(Y[Y[:,i]==1,:])

In [40]:
balans_len = [len(yc) for yc in Y_clus_list]
min_len = min(balans_len)
min_len, balans_len

(998, [1000, 1000, 1000, 999, 998])

In [41]:
X = np.concatenate([xc[:min_len] for xc in X_clus_list])
Y = np.concatenate([yc[:min_len] for yc in Y_clus_list])

In [42]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [45]:
i = 0
aas = [0.0001, 0.1]
miis = [2000, 5000, 10000]
modds = [( 50, 50, 50, 50), (100, 100, 100, 100), (50, 100, 50, 25)]
df = pd.DataFrame(columns = ['alpha','max_iter','model', 'train_acc','test_acc','train_time'])
for a in aas:
    for mi in miis:
        for mod in modds:
            st = time()
            mlp = MLPClassifier(hidden_layer_sizes=mod, alpha=a, max_iter=mi)
            mlp.fit(X_train, y_train)
            end = time() - st
            
            acc_tr = accuracy_score(y_train, mlp.predict(X_train)) # Train Accuracy
            acc = accuracy_score(y_test, mlp.predict(X_test)) # Test Accuracy
            df.loc[i] = [a, mi, mod, acc_tr,acc,end]
            i=i+1

In [46]:
df # Results
# 0	0.00001	2000	(50, 50, 50, 50)	0.765271	0.187783	9.691846 (43)
# 	0.00001	2000	(50, 50, 50, 50)	0.75	0.192308	9.430116 (44)
# 0.00001	2000	(50, 50, 50, 50)	0.616516	0.199095	10.068232 (45)
# 0.00001	2000	(50, 50, 50, 50)	0.796946	0.20362	6.908683 (46)

,alpha,max_iter,model,train_acc,test_acc,train_time
0,0.0001,2000,"(50, 50, 50, 50)",0.638527,0.278557,30.240829
1,0.0001,2000,"(100, 100, 100, 100)",0.903808,0.297595,36.590581
2,0.0001,2000,"(50, 100, 50, 25)",0.615481,0.284569,26.710021
3,0.0001,5000,"(50, 50, 50, 50)",0.581914,0.249499,24.863872
4,0.0001,5000,"(100, 100, 100, 100)",0.872244,0.318637,30.948753
5,0.0001,5000,"(50, 100, 50, 25)",0.665581,0.261523,57.249285
6,0.0001,10000,"(50, 50, 50, 50)",0.659319,0.277555,38.573423
7,0.0001,10000,"(100, 100, 100, 100)",0.919589,0.308617,34.071180
8,0.0001,10000,"(50, 100, 50, 25)",0.722194,0.258517,53.968146
9,0.1000,2000,"(50, 50, 50, 50)",0.143036,0.133267,18.524315


In [47]:
df["train_acc"].max()
df[df["train_acc"] == df["train_acc"].max()]

,alpha,max_iter,model,train_acc,test_acc,train_time
7,0.0001,10000,"(100, 100, 100, 100)",0.919589,0.308617,34.07118


In [48]:
df[df["test_acc"] == df["test_acc"].max()]

,alpha,max_iter,model,train_acc,test_acc,train_time
4,0.0001,5000,"(100, 100, 100, 100)",0.872244,0.318637,30.948753


In [49]:
df.sort_values("test_acc",  ascending=False)[:15]

,alpha,max_iter,model,train_acc,test_acc,train_time
4,0.0001,5000,"(100, 100, 100, 100)",0.872244,0.318637,30.948753
7,0.0001,10000,"(100, 100, 100, 100)",0.919589,0.308617,34.071180
1,0.0001,2000,"(100, 100, 100, 100)",0.903808,0.297595,36.590581
2,0.0001,2000,"(50, 100, 50, 25)",0.615481,0.284569,26.710021
13,0.1000,5000,"(100, 100, 100, 100)",0.641283,0.280561,35.530849
0,0.0001,2000,"(50, 50, 50, 50)",0.638527,0.278557,30.240829
6,0.0001,10000,"(50, 50, 50, 50)",0.659319,0.277555,38.573423
10,0.1000,2000,"(100, 100, 100, 100)",0.527054,0.274549,30.687349
5,0.0001,5000,"(50, 100, 50, 25)",0.665581,0.261523,57.249285
8,0.0001,10000,"(50, 100, 50, 25)",0.722194,0.258517,53.968146


In [52]:
max_acc = 0.0
for i in range(20):
    mlp = MLPClassifier(hidden_layer_sizes=(100, 100, 100, 100), alpha=0.0001, max_iter=5000)
    mlp.fit(X_train, y_train)
    
    
    acc_tr = accuracy_score(y_train, mlp.predict(X_train)) # Train Accuracy
    acc = accuracy_score(y_test, mlp.predict(X_test)) # Test Accuracy
    print(f"{i}"+" "*10, end="\r")
    if max_acc < acc:
        max_acc = acc
        print(acc_tr, acc)
        pickle.dump(mlp, open(name_model+".sav", 'wb')) 

0.874749498997996 0.31763527054108215
0.8512024048096193 0.3226452905811623
0.8627254509018036 0.32364729458917835


/home/daniil/program/python/project/layout_highlighter/imgDoc/env/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:698: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


/home/daniil/program/python/project/layout_highlighter/imgDoc/env/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:698: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


/home/daniil/program/python/project/layout_highlighter/imgDoc/env/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:698: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


/home/daniil/program/python/project/layout_highlighter/imgDoc/env/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:698: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


/home/daniil/program/python/project/layout_highlighter/imgDoc/env/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:698: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


In [53]:
mlp.n_iter_

2